In [2]:
def sigma(self,n, chow=False):
    print(self, n)
    d = self.degree()
    N = self.domain().dimension_relative()
    F2 = self.nth_iterate_map(n)
    P2 = F2.domain()
    T = 1
    f = F2
    for j in range(N,-1,-1):
        print("j:",j)
        X = f.periodic_points(1,return_scheme=True, minimal=False)
        Xa = X.affine_patch(j)
        fa = f.dehomogenize(j)
        Pa = fa.domain()
        if chow:
            R = PolynomialRing(self.base_ring(),'v', N+N+3, order='lex')
            im = [R.gen(i) for i in range(j)] + (N-j)*[0] + [R.gen(i) for i in range(N,R.ngens())]
            R_zero = {R.gen(N):1}
            for j in range(N+1,N+N+1):
                R_zero.update({R.gen(j):0})
            vars = list(R.gens())
            t = vars.pop()
            w = vars.pop()
            vars = vars[:N]
        else:
            R = PolynomialRing(self.base_ring(),'v', N+2, order='lex')
            im = list(R.gens())[:j] + (N-j)*[0] + [R.gen(i) for i in range(N,R.ngens())]
            vars = list(R.gens())
            t = vars.pop()
            w = vars.pop()
        print("im:",im)
        
        Ra = Pa.coordinate_ring()
        phi = Ra.hom(Ra.gens(),R)
        MS = MatrixSpace(R,N,N)
        M = t*MS.one()

        g = (M-jacobian([phi(F.numerator())/phi(F.denominator()) for F in fa], vars)).det()
        if chow:
            I = R.ideal([phi(h)(im) for h in Xa.defining_polynomials()] \
                        + [w*R(g.denominator())(im)-R(g.numerator())(im) \
                           + sum(R.gen(j-1)*R.gen(N+j)*(R(g.denominator())(im)) for j in range(1,N+1))])
        else:
            I = R.ideal([phi(h)(im) for h in Xa.defining_polynomials()] \
                        + [w*R(g.denominator())(im)-R(g.numerator())(im)])
        G = I.groebner_basis()
        T = T*G[-1]

        if chow:
            T2 = T.specialization(R_zero)
            newR = PolynomialRing(self.base_ring(), 'w,t',2, order='lex')
            psi = T2.parent().hom(N*[0]+list(newR.gens()), newR)
            print(psi(T2).degrees())
            #print psi(T2)
        else:
            T2 = T
            newR = PolynomialRing(self.base_ring(), 'w,t',2, order='lex')
            print(T2.parent())
            psi = T2.parent().hom(N*[0]+list(newR.gens()), newR)
            print(psi)
            print(psi(T2).degrees())
            #print psi(T2)
    if chow:
        T = T.specialization(R_zero)
    return psi(T)

In [8]:
P.<x,y,z>=ProjectiveSpace(QQ,2)
f = DynamicalSystem([x^2+3*y^2,y^2,z^2])
sigma(f,1,chow=True)

Dynamical System of Projective Space of dimension 2 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z) to
        (x^2 + 3*y^2 : y^2 : z^2) 1
j: 2
im: [v0, v1, v2, v3, v4, v5, v6]
(4, 8)
j: 1
im: [v0, 0, v2, v3, v4, v5, v6]
(6, 12)
j: 0
im: [0, 0, v2, v3, v4, v5, v6]
(7, 14)


1/3*w^7 - 7/3*w^6*t^2 + 10/3*w^6*t - 4/3*w^6 + 7*w^5*t^4 - 20*w^5*t^3 + 28*w^5*t^2 - 24*w^5*t + 16*w^5 - 35/3*w^4*t^6 + 50*w^4*t^5 - 120*w^4*t^4 + 560/3*w^4*t^3 - 176*w^4*t^2 + 64*w^4*t + 35/3*w^3*t^8 - 200/3*w^3*t^7 + 680/3*w^3*t^6 - 1520/3*w^3*t^5 + 2096/3*w^3*t^4 - 1760/3*w^3*t^3 + 256*w^3*t^2 - 7*w^2*t^10 + 50*w^2*t^9 - 220*w^2*t^8 + 640*w^2*t^7 - 1200*w^2*t^6 + 1536*w^2*t^5 - 1216*w^2*t^4 + 384*w^2*t^3 + 7/3*w*t^12 - 20*w*t^11 + 108*w*t^10 - 1160/3*w*t^9 + 928*w*t^8 - 1568*w*t^7 + 1664*w*t^6 - 768*w*t^5 - 1/3*t^14 + 10/3*t^13 - 64/3*t^12 + 272/3*t^11 - 800/3*t^10 + 1664/3*t^9 - 704*t^8 + 384*t^7

In [96]:
R.<v0,v1,v2,v3>=QQbar[]
I=R.ideal([v0^2*v1 - v0, v0^2 - v0*v1^2, -v1^3 + 1,(v0*v2 + v1*v3)])
I.elimination_ideal([v0,v1]).gen(0).factor()

v3 * (v2 + (-0.50000000000000000? - 0.866025403784439?*I)*v3) * (v2 + (-0.50000000000000000? + 0.866025403784439?*I)*v3) * (v2 + v3)

In [ ]:
P.<x,w,y,z>=ProjectiveSpace(QQ,3)
f = DynamicalSystem([x^2,w^2,z^2,y^2])
sigma(f,1,chow=True)

In [6]:
P.<x,w,y,z>=ProjectiveSpace(QQ,3)
f = DynamicalSystem([x^2,z^2+z*w,w^2,y^2])
J = jacobian(f.defining_polynomials(),P.gens())
for Q in f.periodic_points(1):
    print(Q,J(list(Q)))
    print("-----")

(1 : 0 : 0 : 0) [2 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
-----


In [12]:
P.<x,y,z>=ProjectiveSpace(QQbar,2)
f = DynamicalSystem([x^2,z^2 + z*y,y^2])
J = jacobian(f.defining_polynomials(),P.gens())
for Q in f.periodic_points(1):
    print(Q,)
    print("-----")

(0 : -0.6623589786223730? - 0.5622795120623013?*I : 1)
-----
(0 : -0.6623589786223730? + 0.5622795120623013?*I : 1)
-----
(0 : 1.324717957244746? : 1)
-----
(0.1225611668766537? - 0.744861766619745?*I : -0.6623589786223730? + 0.5622795120623013?*I : 1)
-----
(0.1225611668766537? + 0.744861766619745?*I : -0.6623589786223730? - 0.5622795120623013?*I : 1)
-----
(1 : 0 : 0)
-----
(1.754877666246693? : 1.324717957244746? : 1)
-----


In [67]:
P.<x,y,z>=ProjectiveSpace(QQ,2)
f = DynamicalSystem([x^2,z^2,y^2])
J = jacobian(f.defining_polynomials(),P.gens())
for Q in f.periodic_points(1):
    print(Q,J(list(Q)))
    print("-----")

(0 : 1 : 1) [0 0 0]
[0 0 2]
[0 2 0]
-----
(1 : 0 : 0) [2 0 0]
[0 0 0]
[0 0 0]
-----
(1 : 1 : 1) [2 0 0]
[0 0 2]
[0 2 0]
-----


In [68]:
sigma(f,1)

Dynamical System of Projective Space of dimension 2 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z) to
        (x^2 : z^2 : y^2) 1
j: 2
[v0, v1, v2, v3]
im: [v0, v1]
[x^2/y^2, 1/y^2]
(-2*v0*v1^3*v3 - 4*v0 + v1^5*v3^2 + 2*v1^2*v3)/v1^5
2*v0*v1^3*v3 + 4*v0 + v1^5*v2 - v1^5*v3^2 - 2*v1^2*v3
Ideal (v0^2*v1 - v0, v0^2 - v0*v1^2, -v1^3 + 1, 2*v0*v1^3*v3 + 4*v0 + v1^5*v2 - v1^5*v3^2 - 2*v1^2*v3) of Multivariate Polynomial Ring in v0, v1, v2, v3 over Rational Field
(2, 4)
j: 1
[v0, v1, v2, v3]
im: [v0, 0]
[x^2/z^2, 1/z^2]
(2, 4)
j: 0
[v0, v1, v2, v3]
im: [0, 0]
[z^2, y^2]
v3^2
v2 - v3^2
Ideal (0, 0, 0, v2 - v3^2) of Multivariate Polynomial Ring in v0, v1, v2, v3 over Rational Field
(3, 6)


w^3 - 3*w^2*t^2 - 2*w^2*t + 4*w^2 + 3*w*t^4 + 4*w*t^3 - 8*w*t^2 - 8*w*t - t^6 - 2*t^5 + 4*t^4 + 8*t^3

In [13]:
(x^5-2*x^4+x^3-x^2).factor()

(x - 1.754877666246693?) * (x - 0.1225611668766537? - 0.744861766619745?*I) * (x - 0.1225611668766537? + 0.744861766619745?*I) * x^2

In [15]:
F=f.dehomogenize(1)
F.multiplier(Q.dehomogenize())

In [ ]:
A.<x,z>=AffineSpace(QQ,2)
t=3
#(x^2- t*z^2)^2,4*x*z*(x-z)*(x-t*z)
f=DynamicalSystem_affine([(x^2- t*z^2)^2/(4*x*z*(x-z)*(x-t*z)), (x^2+z^2)^2/(4*x*z*(x-z)*(x-t*z))])
F=f.homogenize(1)
F.is_morphism()

In [64]:
sigma(f,1)

Dynamical System of Projective Space of dimension 2 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z) to
        (x^2 : z^2 : y^2) 1
j: 2
[v0, v1, v2, v3]
im: [v0, v1]
[x^2/y^2, 1/y^2]
(-2*v0*v1^3*v3 - 4*v0 + v1^5*v3^2 + 2*v1^2*v3)/v1^5
2*v0*v1^3*v3 + 4*v0 + v1^5*v2 - v1^5*v3^2 - 2*v1^2*v3
Ideal (v0^2*v1 - v0, v0^2 - v0*v1^2, -v1^3 + 1, 2*v0*v1^3*v3 + 4*v0 + v1^5*v2 - v1^5*v3^2 - 2*v1^2*v3) of Multivariate Polynomial Ring in v0, v1, v2, v3 over Rational Field
(2, 4)
j: 1
[v0, v1, v2, v3]
im: [v0, 0]
[x^2/z^2, 1/z^2]
(2, 4)
j: 0
[v0, v1, v2, v3]
im: [0, 0]
[z^2, y^2]
v3^2
v2 - v3^2
Ideal (0, 0, 0, v2 - v3^2) of Multivariate Polynomial Ring in v0, v1, v2, v3 over Rational Field
(3, 6)


w^3 - 3*w^2*t^2 - 2*w^2*t + 4*w^2 + 3*w*t^4 + 4*w*t^3 - 8*w*t^2 - 8*w*t - t^6 - 2*t^5 + 4*t^4 + 8*t^3

In [39]:
Y=f.periodic_points(1,return_scheme=True)
Y.affine_patch(2).defining_ideal()

Ideal (x^2*y - x*y - x, -x*y^2 + x^2, -y^3 + y + 1) of Multivariate Polynomial Ring in x, y over Algebraic Field

In [53]:
R.<v0,v1,v2,v3>=QQ[]
I=R.ideal([v0^2 - v0*v2^2 + 2*v0*v2*v3 - v0*v3^2, v1 + v2 - v3, v2^3 + (-3)*v2^2*v3 + 3*v2*v3^2 - v2 - v3^3 + v3 + 1])
I=R.ideal([v0^2*v1 - v0*v1 - v0, v0^2 - v0*v1^2, -v1^3 + v1 + 1, v1 + v2 - v3])
I=R.ideal([v0^2*v1 - v0*v1 - v0, v0^2 - v0*v1^2, -v1^3 + v1 + 1, v1 + v2 - v3])
I.elimination_ideal([v0,v1]).gens()[0].factor()

(-1) * (-v2^3 + 3*v2^2*v3 - 3*v2*v3^2 + v3^3 + v2 - v3 - 1)

In [54]:
(x^3-x-1).factor()

(x - 1.324717957244746?) * (x + 0.6623589786223730? - 0.5622795120623013?*I) * (x + 0.6623589786223730? + 0.5622795120623013?*I)

In [59]:
def sigma(self,n, chow=False):
    print(self, n)
    d = self.degree()
    N = self.domain().dimension_relative()
    F2 = self.nth_iterate_map(n)
    P2 = F2.domain()
    T = 1
    f = F2
    for j in range(N,-1,-1):
        print("j:",j)
        X = f.periodic_points(1,return_scheme=True, minimal=False)
        Xa = X.affine_patch(j)
        fa = f.dehomogenize(j)
        Pa = fa.domain()
        if chow:
            R = PolynomialRing(self.base_ring(),'v', N+N+3, order='lex')
            im = [R.gen(i) for i in range(j)] + (N-j)*[0]
            R_zero = {R.gen(N):1}
            for j in range(N+1,N+N+1):
                R_zero.update({R.gen(j):0})
            #print R_zero
            vars = list(R.gens())
            t = vars.pop()
            w = vars.pop()
            vars = vars[:N]
        else:
            R = PolynomialRing(self.base_ring(),'v', N+2, order='lex')
            im = list(R.gens())[:j] + (N-j)*[0]
            vars = list(R.gens())
            t = vars.pop()
            w = vars.pop()
        print("im:",im)
        #print t,w

        phi = Pa.coordinate_ring().hom(im,R)
        #print phi
        MS = MatrixSpace(R,N,N)
        M = t*MS.one()
        print(list(fa))
        g = (M-jacobian([phi(F.numerator())/phi(F.denominator()) for F in fa], vars)).det()
        #print "g:",g
        if chow:
            I = R.ideal([phi(h) for h in Xa.defining_polynomials()] + [w*g.denominator()-R.gen(N)*g.numerator() + sum(R.gen(j-1)*R.gen(N+j)*g.denominator() for j in range(1,N+1))])
        else:
            I = R.ideal([phi(h) for h in Xa.defining_polynomials()] + [w*g.denominator()-g.numerator()])
        #print I
        G = I.groebner_basis()
        #print "G:",G[-1]
        T = T*G[-1]
        #print "T:",T
        #print T.coefficient(R.gen(2*N+1)**8)
        #print T.specialization(R_zero)
        #print T
        if chow:
            T2 = T.specialization(R_zero)
            newR = PolynomialRing(self.base_ring(), 'w,t',2, order='lex')
            psi = T2.parent().hom(N*[0]+list(newR.gens()), newR)
            print(psi(T2).degrees())
            #print psi(T2)
        else:
            T2 = T
            newR = PolynomialRing(self.base_ring(), 'w,t',2, order='lex')
            psi = T2.parent().hom(N*[0]+list(newR.gens()), newR)
            print(psi(T2).degrees())
            #print psi(T2)
    if chow:
        T = T.specialization(R_zero)
    return psi(T)

def num_sigma(self, n):
    fc = self.change_ring(CC)
    per= self.periodic_points(n,minimal=False,return_scheme=True).rational_points(F=CC)
    mul = []
    t=1
    for Q in per:
        t=t*fc.multiplier(Q,n,check=False).characteristic_polynomial()
        c=fc.multiplier(Q,n, check=False).characteristic_polynomial().coefficients(sparse=False)
        c.pop(-1) #always monic
        mul.append(c)
    sig=[]
    e = SymmetricFunctions(CC).e()

    N = len(mul)
    R = CC
    for i in range(0,N):
        T=[]
        for k in range(len(mul[0])):
            T.append((R(e([i+1]).expand(N)([m[k] for m in mul]))))
        sig.append(T)
    return sig,t

In [62]:
P.<x,y,z>=ProjectiveSpace(QQ,2)
f=DynamicalSystem([x^2,z^2,y^2])
sigma(f,1)

Dynamical System of Projective Space of dimension 2 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z) to
        (x^2 : z^2 : y^2) 1
j: 2
im: [v0, v1]
[x^2/y^2, 1/y^2]
(2, 4)
j: 1
im: [v0, 0]
[x^2/z^2, 1/z^2]


ZeroDivisionError: 

In [2]:
P.<x,y>=ProjectiveSpace(QQ,1)
f=DynamicalSystem([x^2-2*x*y,y^2])
#f=DynamicalSystem([x^4-2*x^3*y-x^2*y^2+3*x*y^3,y^4])
#for Q in f.periodic_points(1):
#    print Q,f.multiplier(Q,1).charpoly().factor()
sigma(f,1)

Dynamical System of Projective Space of dimension 1 over Rational Field
  Defn: Defined on coordinates by sending (x : y) to
        (x^2 - 2*x*y : y^2) 1
j: 1
im: [v0]
[x^2 - 2*x]
(2, 2)
j: 0
im: [0]
[y^2/(-2*y + 1)]
(3, 3)


w^3 - 3*w^2*t + 2*w^2 + 3*w*t^2 - 4*w*t - 8*w - t^3 + 2*t^2 + 8*t

In [8]:
R.<w>=QQ[]

(w-1)*(w-2)*(w-3)

w^3 - 6*w^2 + 11*w - 6

In [11]:
R.<w,t>=QQ[]

((w-(t-1))*(w-(t-2))*(w-(t-3))).specialization({w:0})

-t^3 + 6*t^2 - 11*t + 6

In [4]:
f.sigma_invariants(1)

[2, -8, 0]

In [3]:
sigma(f,1)

Dynamical System of Projective Space of dimension 1 over Rational Field
  Defn: Defined on coordinates by sending (x : y) to
        (x^2 - 2*x*y : y^2) 1
j: 1
im: [v0]
[x^2 - 2*x]
(2, 2)
j: 0
im: [0]
[y^2/(-2*y + 1)]
(3, 3)


w^3 - 3*w^2*t + 2*w^2 + 3*w*t^2 - 4*w*t - 8*w - t^3 + 2*t^2 + 8*t

In [11]:
f.sigma_invariants?

In [34]:
P.<x,y>=ProjectiveSpace(QQ,1)
f=DynamicalSystem([x^3,y^3])
for Q in f.periodic_points(1):
    print Q,f.multiplier(Q,1).charpoly().factor()

(-1 : 1) x - 3
(0 : 1) x
(1 : 0) x
(1 : 1) x - 3


In [14]:
(x-1)*x*(x+1)*(x-2) + x

x^4 - 2*x^3 - x^2 + 3*x